In [1]:
#!pip3 install --upgrade pip

#!pip3 install re
#!pip3 install time
#!pip3 install urllib3
#!pip3 install requests
#!pip3 install logging
#!pip3 install pandas
#!pip3 install tqdm
#!pip3 install beautifulsoup4
#!pip3 install selenium-stealth
#!pip3 install fake-useragent
#!pip3 install selenium


In [3]:
import re
import time
import urllib3
import requests
import logging
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime
from selenium_stealth import stealth
from fake_useragent import UserAgent
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException

# отключаем предупреждения
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# настройка логирования для подавления предупреждений
logging.getLogger("bs4").setLevel(logging.CRITICAL)


In [3]:
# класс парсера сайтов
class SiteContentParserDSP:

    # инициализатор
    def __init__(self):
        # список исключений для отбраковки сайтов по причине недоступности сайта или капчи перед доступом к сайту
        self.content_exceptions = ['your request has been denied','проверяем, человек ли вы. это может занять несколько секунд.','сайт заблокирован хостинг-провайдером','error 404 (not found)!!1 404. that’s an error.', \
                                   'подтвердите, что вы человек, выполнив указанное действие.', 'если вы человек, нажмите на похожий цвет', 'антиспам:я не робот', '.err_name_not_resolved', 'err_timed_out', 'нажмите, если вы человек перейдите по ссылке', \
                                   'домен не прилинкован ни к одной из директорий на сервере!', 'проверка recaptcha пожалуйста, пройдите проверку', 'success! success! your new web server is ready to use.', \
                                   'he website has been stopped sorry, this site has been stopped by the administrator', 'parse error: syntax error, unexpected', 'this site is currently under construction. please check back soon', \
                                   'сайт отключен, технические работы сайт был удалён, из-за взлома','пожалуйста, включите javaScript, чтобы продолжить.', 'this site is currently under construction. please check back soon. domain: hd-lord.net', \
                                   'the server encountered an internal error or misconfiguration and was unable to complete your request.', 'сайт оффлайн приносим вам свои извинения за доставленные неудобства']

    # функция доступа к контенту сайта
    def content_soup(self, site_content):
        # проверка успешности запроса
        if site_content.status_code == 200:
            # извлекаем html код с переданой страницы
            soup = BeautifulSoup(site_content.content, 'html.parser')
        else:
            # если переданный сайт не доступен то пердаем пустой контент
            soup = BeautifulSoup('', 'html.parser')
            # возвращаем объект формата BeautifulSoup
        return soup

    # функция очистки текста
    def cleaner(self, text):
        # условие проверки объекта на существование
        if text:
            # убираем лишний html синтаксис
            text_first_clean = text.strip().replace('\n', ' ').replace('\r', ' ').replace('\xa0', ' ')
            # убираем лишние пробелы
            text_second_clean = re.sub(r'\s+', ' ', text_first_clean).strip()
            # возврат очищенного текста
            return text_second_clean
        else:
            # возвращаем пустой объект
            return None

    # функция получения контента страниц с использованием BeautifulSoup
    def content_site_beautifulsoup(self, content_soup, url):
        # обявляем пустой словарь для итератора
        site_content = {}
        # добавляем номенование сайта в словарь
        site_content['site'] = url
        # получаем тайтл которым именуется страница
        site_content['title'] = self.cleaner(content_soup.head.title.string) if content_soup.head and content_soup.head.title else None
        # получаем описание страницы из раздела мета
        site_content['description'] = self.cleaner(content_soup.find('meta', attrs={'name': 'description'})['content']) \
            if content_soup.find('meta', attrs={'name': 'description'}) and 'content' in content_soup.find('meta', attrs={'name': 'description'}).attrs else None
        # получаем ключевые слова страницы из раздела мета
        site_content['keywords'] = self.cleaner(content_soup.find('meta', attrs={'name': 'keywords'})['content']) \
            if content_soup.find('meta', attrs={'name': 'keywords'}) and 'content' in content_soup.find('meta', attrs={'name': 'keywords'}).attrs else None
        # получаем тайтл страницы из раздела мета
        site_content['title_meta'] = self.cleaner(content_soup.find('meta', attrs={'property': 'og:title'})['content']) \
            if content_soup.find('meta', attrs={'property': 'og:title'}) and 'content' in content_soup.find('meta', attrs={'property': 'og:title'}).attrs else None
        # получаем контент только блока main div как правило это тело страницы за исключением хедера футера и боковых меню
        #site_content['content_main'] = self.cleaner(content_soup.select_one('body > main').get_text()) if content_soup.select_one('body > main') else None
        # получаем текст всего контента страницы
        site_content['full_content'] = self.cleaner(content_soup.get_text()) if content_soup.get_text() != '' else None
        # сохраняем оригинал кода страницы без предобработок (возможно будет полезен сайнтистам)
        site_content['site_html'] = content_soup if content_soup else None
        # возвращаем заполненный словарь
        return site_content

    # функция очистки адресов
    def url_transformer(self, url):
        # переменная исключаемого слова
        trash_url_part = '.turbopages.org'
        # возвращаем или очищенный адрес в случае присутствиия мусорной части или исходный адрес
        return url.replace(trash_url_part, "").replace('-', '.').replace('..', '-') if trash_url_part in url else url

    # функция проверки содержания ошибки в контенте когда сайт не вернул код ошибки но выдал страницу ошибки
    def site_excluded(self, content):
        # используем .get() для безопасного доступа к значению
        full_content = content.get('full_content', '').lower() if content.get('full_content', '') else ''
        # проверяем, если full_content пуст или содержит одно из исключений
        if full_content == '' or any(exception in full_content for exception in self.content_exceptions):
            # возвращаем значения правда, сайт сиключаемый
            return True
        # возвращаем значения лож, сайт несиключаемый
        return False

    # функция сбора данных по сыллкам с использованием BeautifulSoup
    def frame_content_searcher_bs(self, site_list):
        # объявляем пустой список для заполнения результата парсинга
        df = []
        # объявляем пустой список сылок доступ к которым не получилось получить для дальнейшей попытки обработки через селениум
        df_for_selen = []
        # цикл перебора списка сайтов
        for url in tqdm(site_list, desc="Парсинг ссылок BeautifulSoup", unit="ссылка"):
            # испоьзуем фейковые юзер аненты для исключения блокеровки
            ua = UserAgent(browsers='chrome')
            # присваиваем рандомный агент
            user_agent = ua.random
            # чистим адрес
            url = self.url_transformer(url)
            # попытка доступа к странице
            try:
                # пробуем получить доступ к странице используя http
                site_content_html = requests.get('http://' + url, verify=False, timeout=10, headers={'User-Agent':user_agent})
                # проверяем на ошибки HTTP
                site_content_html.raise_for_status()
            # отрабатываем исключение
            except requests.RequestException:
                try:
                    # пробуем получить доступ к странице используя https
                    site_content_html = requests.get('https://' + url, verify=False, timeout=10, headers={'User-Agent':user_agent})
                    # проверяем на ошибки HTTP
                    site_content_html.raise_for_status()
                except requests.RequestException:
                    # добавляем сылку доступ к которой не получилось получить в список для отработки селениумом
                    df_for_selen.append(url)
                    # прекращаем выполнения кода в данной итерации и переходим к отработки следующещей сылки
                    continue   
            # получаем содержимое страницы
            site_content_text = self.content_soup(site_content_html)
            # получаем данные собранные со страницы
            content = self.content_site_beautifulsoup(site_content_text , url)
            # проверяем найденный контент на содержание не явных ошибок
            if self.site_excluded(content):
                # добавляем сылку доступ к которой не получилось получить в список для отработки селениумом
                df_for_selen.append(url)
                # прекращаем выполнения кода в данной итерации и переходим к отработки следующещей сылки
                continue
            # добавляем результат сбора данных в спиок
            df.append(content)
        # возвращем кортеж из двух результирующих списков
        return df, df_for_selen

    # функция инициализации вебдрайвера с заданными параметарми
    def webdriver(self):
        # инициализируем юзер агент для сокрытия реальных данных браузера для избежания блокировки
        ua = UserAgent(browsers='chrome')
        # берем рандомный параметр пользовательского агента
        user_agent = ua.random
        # инициализируем оъект опций драйвера
        options = Options()
        # режим работы в фоновом режиме
        options.add_argument('--headless=new')
        # присваивание выбранного пользовательского агента 
        options.add_argument(f'--user-agent={user_agent}')
        # задаем максимальный размер окна чтобы сайт не переходил на работу в режиме мобильной версии в которой элементы не подгружаются тк расположены иначе
        options.add_argument('start-maximized')
        # присваеваем окну конкретный размер
        options.add_argument('--window-size=1920,1080')
        # задаем параметр отработки графики только на процессоре
        options.add_argument('--disable-gpu')
        # задаем параметр отключение хранения временных файлов
        options.add_argument('--disable-dev-shm-usage')
        # отключение уведомлений в браузере
        options.add_argument('--disable-notifications')
        # отключаем блокировку всплывающих окон
        options.add_argument('--disable-popup-blocking')
        # отключаем режим песочницы
        options.add_argument('--no-sandbox')
        # активация использования браузеру джава скрипта
        options.add_argument('--enable-javascript')
        # отклчение детекции джава скриптом браузера как браузера под автоматическоим управлением
        options.add_argument( '--disable-blink-features=AutomationControlled')
        # добавляется экспериментальная опция, исключающая переключатель "enable-automation", чтобы предотвратить автоматическое обнаружение, что браузер управляется вебдрайвером
        options.add_experimental_option('excludeSwitches', ['enable-automation'])
        # добавляется экспериментальная опция, отключающая расширение автоматизации в браузере, что также помогает скрыть использование вебдрайвера
        options.add_experimental_option('useAutomationExtension', False)
        # инициализируем вебдрайвер с заданными ранее опциями
        driver = Chrome(options=options)                               
        # настройка селениум стелс режим для анаоноимности браузера
        stealth(driver,
                user_agent=user_agent,
                languages=['ru-RU', 'ru'],
                vendor='Google Inc.',
                platform=user_agent.split(' ')[1].strip('()'),
                webgl_vendor='Intel Inc.',
                renderer='Intel Iris OpenGL Engine',
                fix_hairline=True,
                run_on_insecure_origins=True
                )
        # замена прототипа браузера для исключения детекции браузера под автоматическим управлением
        driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
            'source':
            '''
            const newProto = navigator.__proto__;
            delete newProto.webdriver;
            navigator.__proto__ = newProto;
            '''
            }
        )
        # удаляем все куки если они были ранее
        driver.delete_all_cookies()
        # Установка размеров окна
        driver.set_window_size(1920, 1080)
        # центровка расположения контента
        driver.set_window_position(0, 0)
        # возвращаем драйвер
        return driver

    # функция сбора данных по сыллкам с использованием Selenium
    def frame_content_searcher_selen(self, site_list):
        # объявляем пустой список для заполнения результата парсинга
        df_selen = []
        # объявляем пустой список сылок доступ к которым не получилось получить для дальнейшей попытки обработки иными способами
        df_for_manual_work = []
        # цикл перебора списка сайтов
        for url in tqdm(site_list, desc="Парсинг ссылок Selenium", unit="ссылка"):
            # создаем экземпляр вебдрайвера
            driver = self.webdriver()
            # попытка доступа к странице
            try:
                # пробуем получить доступ к странице используя http передаем url страницы в драйвер
                driver.get('http://' + url)
                # не явное ожидание загрузки страницы 
                driver.implicitly_wait(5)
                # скролим страницу до конца вверх чтобы загрузились все элементы с которыми будем взаимодействовать
                driver.execute_script("window.scrollTo(0, 0);")
                # явное ожидание для загрузки данных
                time.sleep(5)
            # отрабатываем исключение
            except WebDriverException:
                try:
                    # пробуем получить доступ к странице используя https передаем url страницы в драйвер
                    driver.get('https://' + url)
                    # не явное ожидание загрузки страницы 
                    driver.implicitly_wait(5)
                    # скролим страницу до конца вверх чтобы загрузились все элементы с которыми будем взаимодействовать
                    driver.execute_script("window.scrollTo(0, 0);")
                    # явное ожидание для загрузки данных
                    time.sleep(5)
                except WebDriverException:
                    # добавляем сылку доступ к которой не получилось получить в список для отработки селениумом
                    df_for_manual_work.append(url)
                    # прекращаем выполнения кода в данной итерации и переходим к отработки следующещей сылки
                    continue
            # передаем содержимое вебдрайвера после взаимодействия со страницей из селениума в формате html
            soup = BeautifulSoup(driver.page_source, 'html.parser')      
            # получаем данные собранные со страницы
            content = self.content_site_beautifulsoup(soup , url)
            # проверяем найденный контент на содержание не явных ошибок
            if self.site_excluded(content):
                # добавляем сылку доступ к которой не получилось получить в список для отработки селениумом
                df_for_manual_work.append(url)
                # прекращаем выполнения кода в данной итерации и переходим к отработки следующещей сылки
                continue
            # добавляем результат сбора данных в спиок
            df_selen.append(content)
            # закрываем драйвер
            driver.quit()
        # возвращем кортеж из двух результирующих списков
        return df_selen, df_for_manual_work
    
    # функция объединяющая сбор данных
    def start_parsing(self, site_list):
        # вызываем парсинг основного списка через BeautifulSoup
        df_bs, df_for_selen_bs = self.frame_content_searcher_bs(site_list)
        # вызываем парсинг списка который не был обработан BeautifulSoup через Selenium
        df_selen, df_for_manual_work = self.frame_content_searcher_selen(df_for_selen_bs)
        # объеденяем списки которые собранные обоими парсерами
        df_result = df_bs + df_selen
        # возвращаем дата фреймы результирующий и тот что не удалось отработать парсерами для ручной отработки в случае капчи или отбраковки сайта как недоступного
        return df_result, df_for_manual_work


# Правила использования парсера

- Парсер позволяет собирать данные возращая кортеж списков, список успешно собранных данных и список сайтов данные которых собрать не удалось. Доступные варианты использования:
    - BeautifulSoup: df_result, df_for_manual_work = SiteContentParserDSP().frame_content_searcher_bs(site_df)
    - Selenium: df_result, df_for_manual_work = SiteContentParserDSP().frame_content_searcher_selen(site_df)
    - BeautifulSoup & Selenium: df_result, df_for_manual_work = SiteContentParserDSP().start_parsing(site_df)
- Рекомендуется использовать вариант BeautifulSoup для повышения колличества собранных данных можно использовать комбенированный метод но это существенно увеличиит время сбора данных и потребуется немного усилий по установке вебдрайвера Chrome в используемую систему для корректной работы библиотеки Selenium. Без этого запуск кода будет не возможен.
- В инициализаторе класса присутствует список content_exceptions содержащий список фраз исключений по которым отбраковывается контент содержащий целиком данную фразу. При необходимости список можно дополнить новыми исключениями для более точной отбраковки сайтов. Данные в список заносить только в нижнем регистре без использования заглавных букв.
- Если по какойто причине количество ссылок слишком велико для сбора то рекомендуется сбор данных батчем например по 1000 сайтов. site_df[:1000]
- Так как цель создания парсера сбор данных для последующей классивифации интересующих сайтов то настоятельно рекомендуется не DDoS-ить сайты поторными запросами. При необходимости сбора данных для класссификации новых сайтов, требуется исключать из выборки для нового парсинга сайты класс и тип контента которых уже определены, это позволит уменьшить время сбора нужных данных, уменьшит нагрузку на собственную сеть и интернет канал, сэкономит ресурсы ПК на котором запускается код и в принципе это будет более этичным поведением в интернет пространстве.

In [4]:
# загружаем ссылки из файла в формате список
site_df = pd.read_excel('dsp_adv_cl-20241011111010.xlsx', skiprows=[0, 2], header=0, usecols=[0])['Domain']


In [5]:
# Запуск парсера с использованием BeautifulSoup
#df_result, df_for_manual_work = SiteContentParserDSP().frame_content_searcher_bs(site_df[:50])

# Запуск парсера с использованием Selenium
#df_result, df_for_manual_work = SiteContentParserDSP().frame_content_searcher_selen(site_df[:10])

# Запуск парсера комбинированным методом с использованием сбора основных страниц BeautifulSoup и сбором более сложных страниц через Selenium
df_result, df_for_manual_work = SiteContentParserDSP().start_parsing(site_df)


Парсинг ссылок Selenium: 100%|██████████| 1250/1250 [4:06:57<00:00, 11.85s/ссылка]   


In [6]:
# сохраняем результирующий фрейм данных в файл текущей датой
pd.DataFrame(df_result).to_csv(f"SiteContentParserDSP_result_{datetime.now().strftime('%d%m%Y')}.csv")
# сохраняем фрейм недоступных сайтов или сайтов с капчей в файл текущей датой
pd.DataFrame(df_for_manual_work).to_csv(f"SiteContentParserDSP_for_manual_work_{datetime.now().strftime('%d%m%Y')}.csv")


In [7]:
#display(pd.DataFrame(df_result))
#display(pd.DataFrame(df_for_manual_work))
#display(pd.DataFrame(df_result)['full_content'][38])


In [4]:
#df = pd.read_csv('SiteContentParserDSP_result_18102024.csv')
#df = df.drop(columns=['site_html'])

In [5]:
display(df)

,Unnamed: 0,site,title,description,keywords,title_meta,full_content,site_html
0,0,vetrenyi.ru,Ветреный (2022) смотреть сериал онлайн бесплат...,Cериал Ветреный смотреть онлайн в хорошем каче...,"Ветреный, сериал, смотреть онлайн, бесплатно, ...",Ветреный (2022) смотреть сериал онлайн бесплат...,Ветреный (2022) смотреть сериал онлайн бесплат...,"<!DOCTYPE html>\n\n<html dir=""ltr"" lang=""ru-RU..."
1,1,netpryshi.ru,Бьюти-Журнал NetPryshi.ru — путь к идеальной коже,"Прыщи, черные точки, фурункулы. Особенности ух...",NaN,NaN,Бьюти-Журнал NetPryshi.ru — путь к идеальной к...,"<!DOCTYPE html>\n\n<html lang=""ru-RU"">\n<head>..."
2,2,happylove.top,Отдых в картинках,Смотрите фото онлайн - Отдых в картинках. Тема:,NaN,NaN,Отдых в картинках Меню ВьетнамГрецияЕгипетИтал...,"<!DOCTYPE html>\n\n<html lang=""ru"">\n<head>\n<..."
3,3,35.ru,Новости Вологды и Вологодской области | 35.ру,Последние свежие новости Вологды и Вологодской...,NaN,Новости Вологды и Вологодской области на сайте...,Новости Вологды и Вологодской области | 35.ру ...,"<!DOCTYPE html>\n\n<html lang=""ru"">\n<head>\n<..."
4,4,split.2pdf.com,Split pdf online. Free pdf splitter to separat...,"Using our online PDF splitter, you can separat...",NaN,2pdf.com provides users with a range of free o...,Split pdf online. Free pdf splitter to separat...,"<!DOCTYPE html>\n<html lang=""en""><head><meta c..."
...,...,...,...,...,...,...,...,...
8923,8923,first-otvet.ru,first-otvet.ru,NaN,NaN,NaN,first-otvet.ru Не удается получить доступ к са...,"<html dir=""ltr"" lang=""ru""><head>\n<meta charse..."
8924,8924,atyrauskaya-oblast.jobcareer.ru,atyrauskaya-oblast.jobcareer.ru,NaN,NaN,NaN,atyrauskaya-oblast.jobcareer.ru Не удается пол...,"<html dir=""ltr"" lang=""ru""><head>\n<meta charse..."
8925,8925,daryo-uz.translate.goog,Google Translate,NaN,NaN,NaN,Google TranslateTranslateCan't translate this ...,"<html><head><link href=""https://www.gstatic.co..."
8926,8926,edaplus.info,edaplus.info,NaN,NaN,NaN,edaplus.info Не удается получить доступ к сайт...,"<html dir=""ltr"" lang=""ru""><head>\n<meta charse..."


In [10]:
#df.to_csv(f"SiteContentParserDSP_result_2_{datetime.now().strftime('%d%m%Y')}.csv")

In [11]:
#df.to_parquet('data.parquet')